In [4]:
import folium
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(coord1, coord2):
    # Haversine distance calculation
    lat1, lon1 = coord1
    lat2, lon2 = coord2

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon/2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    R = 6371000  # Earth's radius in meters
    distance = R * c

    return distance

def rotate_point(point, center, angle):
    # Rotation of the point around the center by a given angle
    x, y = point
    cx, cy = center
    rotated_x = (x - cx) * cos(angle) - (y - cy) * sin(angle) + cx
    rotated_y = (x - cx) * sin(angle) + (y - cy) * cos(angle) + cy
    return rotated_x, rotated_y

def generate_circle_centers(top_left, top_right, bottom_right, bottom_left, circle_radius_meters, rotation_angle):
    # Code to generate circle centers goes here
    width = haversine_distance(top_left, top_right)
    height = haversine_distance(top_left, bottom_left)

    num_circles_x = int(width / (2 * circle_radius_meters))
    num_circles_y = int(height / (2 * circle_radius_meters))

    circle_locations = []
    for i in range(num_circles_x + 1):
        for j in range(num_circles_y + 1):
            # Calculate the latitude offset in degrees for each circle
            lat_offset_meters = j * (height / num_circles_y)
            lat_offset_degrees = lat_offset_meters / 111111

            # Calculate the new latitude for the circle center
            circle_center_lat = top_left[0] - lat_offset_degrees

            # Calculate the longitude offset in degrees for each circle
            lon_offset_meters = i * (width / num_circles_x)
            lon_offset_degrees = lon_offset_meters / (111111 * cos(radians(circle_center_lat)))

            # Calculate the new longitude for the circle center
            circle_center_lon = top_left[1] + lon_offset_degrees

            circle_center = (circle_center_lat, circle_center_lon)

            # Rotate the circle center around the top left corner
            rotated_circle_center = rotate_point(circle_center, top_left, rotation_angle)

            circle_locations.append(rotated_circle_center)

    return circle_locations

# Coordinates for the rectangle
top_left = (40.724353, -73.992521)
top_right = (40.724353, -73.980233)
bottom_right = (40.716773, -73.980233)
bottom_left = (40.716773, -73.992521)

# Radius of the circles in meters
circle_radius_meters = 35.405  # Convert 0.022 miles to meters (1 mile is approximately 1609.34 meters)

# Rotation angle in radians
rotation_angle = radians(17)

# Generate circle centers
circle_locations = generate_circle_centers(top_left, top_right, bottom_right, bottom_left, circle_radius_meters, rotation_angle)

# Create a map centered around the midpoint of the rectangle
center_lat = (top_left[0] + bottom_right[0]) / 2
center_lng = (top_left[1] + bottom_right[1]) / 2
map_obj = folium.Map(location=[center_lat, center_lng], zoom_start=15)

# Draw the rotated rectangle in blue
rotated_top_right = rotate_point(top_right, top_left, rotation_angle)
rotated_bottom_right = rotate_point(bottom_right, top_left, rotation_angle)
rotated_bottom_left = rotate_point(bottom_left, top_left, rotation_angle)

folium.Polygon(
    locations=[top_left, rotated_top_right, rotated_bottom_right, rotated_bottom_left],
    color='blue',
    fill=True,
    fill_color='blue'
).add_to(map_obj)

# Draw the rotated circles in red
for circle_center in circle_locations:
    folium.Circle(
        location=circle_center,
        radius=48,
        color='red',
        fill=True,
        fill_color='red'
    ).add_to(map_obj)

# Display the map
map_obj
